### Objective

* Create chunnks of text which we will pass to an LLM for generating IFT data
* We have complex PDF docs which need to be parsed.
* The documents have tables and images which makes the task more complex.
* Explore llamaparse and unstructured both

https://github.com/run-llama/llama_parse/blob/main/examples/demo_json.ipynb
https://docs.cloud.llamaindex.ai/llamaparse/features/metadata


In [1]:
import nest_asyncio
nest_asyncio.apply()

import os
from llama_parse import LlamaParse

In [2]:
!python --version

Python 3.12.2


In [3]:
# check for key
LLAMAPARSE_API_KEY = os.environ.get('LLAMAPARSE_API_KEY')
if LLAMAPARSE_API_KEY is not None:
    print('API key found')
else:
    print('Check for API key in environment variable')

API key found


In [4]:

# instantiate parser
parser = LlamaParse(verbose=True,
                     api_key=LLAMAPARSE_API_KEY,
                     language='en',
                     # result_type="markdown", # or text; no json
                     parsing_instruction="You are parsing a quarterly report from a listed company. Please extract the tables containing financials and other information as well as images."
                     )

In [5]:
json_obs = parser.get_json_result('../data/uber_10q_march_2022.pdf')

Started parsing the file under job_id cac11eca-efd4-4727-b54e-90e696f831a8


In [6]:
# parser returns a list of length 1
type(json_obs), len(json_obs)

(list, 1)

In [7]:
# list item is a dictionary
json_obs[0].keys()

dict_keys(['pages', 'job_metadata', 'job_id', 'file_path'])

In [8]:
# response is in 0th element of the list
# 0th element is a dictionary with parsed content in 'pages' key
type(json_obs[0]['pages']),len(json_obs[0]['pages'])

(list, 106)

In [9]:
# each element of pages is a dictionary
# each page elements in a separate dictionary
    # page - page number
    # text - parsed text
    # md - markdown 
    # images - images on page
type(json_obs[0]['pages'][0]), json_obs[0]['pages'][0].keys()

(dict, dict_keys(['page', 'text', 'md', 'images', 'items']))

In [10]:
# explore how pages with image and tables are stored
# since zero indexing;
json_obs[0]['pages'][43].keys()

dict_keys(['page', 'text', 'md', 'images', 'items'])

In [11]:
# page number
json_obs[0]['pages'][43]['page']

44

In [12]:
# raw text - also includes text from tables
print(json_obs[0]['pages'][43]['text'])

                                                                                        Trips (in millions)
                                                                                                                                1,641               1,769              1,713
                                                                     1,.443              1.447              1511
                                                 1.184
                               737
                            Q2 2020             Q3 2020            Q4 2020             Q1 2021            Q2 2021             Q3 2021            04 2021             Q1 2022
     Gross Bookings. We define Gross Bookings as the total dollar value, including any applicable taxes, tolls, and fees, of: Mobility and New Mobility rides;
Delivery orders (in each case without any adjustment for consumer discounts and refunds); Driver and Merchant earnings; Driver incentives; and Freight revenue.
Gross Bookings do not in

In [13]:
# markdown text - only has tables
print(json_obs[0]['pages'][43]['md'])

#

# Quarterly Report

# Financial Data

**Gross Bookings (in millions)**
| |Q2 2020|Q3 2020|Q4 2020|Q1 2021|Q2 2021|Q3 2021|Q4 2021|Q1 2022|
|---|---|---|---|---|---|---|---|---|
|Mobility|$3,046|$5,905|$6,789|$6,773|$8,640|$9,883|$11,340|$10,723|
|Delivery|$6,961|$8,550|$10,050|$12,461|$12,912|$12,828|$13,444|$13,903|
|Freight|$212|$290|$313|$302|$348|$402|$1,082|$1,823|
|All Other|$5|-|-|-|-|-|-|-|

Take Rate is an operating metric and defined as revenue as a percentage of Gross Bookings.

**Adjusted EBITDA (in millions)**
| |Three Months Ended March 31, 2021|Three Months Ended March 31, 2022|% Change|
|---|---|---|---|
|Adjusted EBITDA|$(359)|$168|**|

** Percentage not meaningful.

Adjusted EBITDA was $168 million, improving $527 million from an Adjusted EBITDA loss of $359 million in the same period in 2021. The improvement was primarily attributable to a $320 million increase in Mobility Adjusted EBITDA, a $230 million improvement in Delivery Adjusted EBITDA, as well as a $31 mi

In [14]:
# 2 images on the page
print(json_obs[0]['pages'][43]['images']), len(json_obs[0]['pages'][43]['images'])

[{'name': 'img_p43_1.png', 'height': 239, 'width': 890}, {'name': 'img_p43_2.png', 'height': 316, 'width': 890}]


(None, 2)

In [15]:
# items - alignment of items on page
print(json_obs[0]['pages'][43]['items'])

[{'type': 'heading', 'lvl': 1, 'value': '', 'md': '#'}, {'type': 'heading', 'lvl': 1, 'value': 'Quarterly Report', 'md': '# Quarterly Report'}, {'type': 'heading', 'lvl': 1, 'value': 'Financial Data', 'md': '# Financial Data'}, {'type': 'text', 'value': '**Gross Bookings (in millions)**', 'md': '**Gross Bookings (in millions)**'}, {'type': 'table', 'rows': [['', 'Q2 2020', 'Q3 2020', 'Q4 2020', 'Q1 2021', 'Q2 2021', 'Q3 2021', 'Q4 2021', 'Q1 2022'], ['Mobility', '$3,046', '$5,905', '$6,789', '$6,773', '$8,640', '$9,883', '$11,340', '$10,723'], ['Delivery', '$6,961', '$8,550', '$10,050', '$12,461', '$12,912', '$12,828', '$13,444', '$13,903'], ['Freight', '$212', '$290', '$313', '$302', '$348', '$402', '$1,082', '$1,823'], ['All Other', '$5', '-', '-', '-', '-', '-', '-', '-']], 'md': '| |Q2 2020|Q3 2020|Q4 2020|Q1 2021|Q2 2021|Q3 2021|Q4 2021|Q1 2022|\n|---|---|---|---|---|---|---|---|---|\n|Mobility|$3,046|$5,905|$6,789|$6,773|$8,640|$9,883|$11,340|$10,723|\n|Delivery|$6,961|$8,550|$10

In [16]:
# see another page - to verify only tables; also has some associated text
# markdown text - only has tables
print(json_obs[0]['pages'][45]['md'])

#

# Uber Technologies, Inc. Quarterly Report

# Adjusted EBITDA Reconciliation

The following table presents a reconciliation of net loss attributable to Uber Technologies, Inc., the most directly comparable GAAP financial measure, to Adjusted EBITDA for each of the periods indicated:

**Adjusted EBITDA reconciliation**
| |Three Months Ended March 31, 2021|Three Months Ended March 31, 2022|
|---|---|---|
|Net loss attributable to Uber Technologies, Inc.|$(108) million|$(5,930) million|
|Net income (loss) attributable to non-controlling interests, net of tax|$(14) million|$12 million|
|Provision for (benefit from) income taxes|$185 million|$(232) million|
|Loss (income) from equity method investments|$8 million|$(18) million|
|Interest expense|$115 million|$129 million|
|Other (income) expense, net|$(1,710) million|$5,557 million|
|Depreciation and amortization|$212 million|$254 million|
|Stock-based compensation expense|$281 million|$359 million|
|Legal, tax, and regulatory reserve ch

##### What does this mean for us?
* Tables are essential since they contain a lot of information about the business
* Tables are getting parsed as text and as markdown
* Repetition will not work for us since questions will be repeated
* We need to explore a hybrid approach - json for images, markdown for text and tables.

In [17]:

# instantiate parser for markdown results
parser = LlamaParse(verbose=True,
                     api_key=LLAMAPARSE_API_KEY,
                     language='en',
                     result_type="markdown", # or text; no json
                     parsing_instruction="You are parsing a quarterly report from a listed company. Please extract the tables containing financials and other information as well as images."
                     )

In [28]:
# parse documents
md_docs = parser.load_data('../data/uber_10q_march_2022.pdf')

Started parsing the file under job_id cac11eca-9353-4239-9155-79c57bf83d0a


In [29]:
# type, etc.
type(md_docs), len(md_docs)

(list, 1)

In [30]:
# since documents is a list with length 1
type(md_docs[0]), print(md_docs[0])

Doc ID: e484169b-cb6e-4e36-8289-f0bc6a10e129
Text: #  # Quarterly Report  # Financial Information  |Title of each
class|Trading Symbol(s)|Name of each exchange on which registered|
|---|---|---| |Common Stock, par value $0.00001 per share|UBER|New
York Stock Exchange|  # SEC Filing Information  Indicate by check mark
whether the registrant (1) has filed all reports required to be filed
by Sectio...


(llama_index.core.schema.Document, None)

In [31]:
# get actual text and tables in markdown; ignores images as seen below
print(md_docs[0].text)

#

# Quarterly Report

# Financial Information

|Title of each class|Trading Symbol(s)|Name of each exchange on which registered|
|---|---|---|
|Common Stock, par value $0.00001 per share|UBER|New York Stock Exchange|

# SEC Filing Information

Indicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 during the preceding 12 months (or for such shorter period that the registrant was required to file such reports), and (2) has been subject to such filing requirements for the past 90 days.

Yes ☒ No ☐

Indicate by check mark whether the registrant has submitted electronically every Interactive Data File required to be submitted pursuant to Rule 405 of Regulation S-T (§232.405 of this chapter) during the preceding 12 months (or for such shorter period that the registrant was required to submit such files).

Yes ☒ No ☐

Indicate by check mark whether the registrant is a large accelerated filer,

##### So what next?

* Markdown response has tables and text both
* We need to extract tables into a separate list and iterate over them separately
* We will use json response to iterate over images.

In [ ]:
# source and GPT result
# https://community.retool.com/t/extract-data-from-markdown-table/28463
# https://chatgpt.com/share/fd555bec-ece4-4336-953c-79c9f4c5cbb2
import re
import pandas as pd

def extract_tables_from_markdown(md_file):
    with open(md_file, 'r', encoding='utf-8') as f:
        md_content = f.read()

    # Regular expression to find Markdown tables
    table_regex = r'\|(.+)\|\n\|?( *[-:]+ *\|)+\n((\|.*\|\n)+)'
    tables = re.findall(table_regex, md_content, re.MULTILINE)

    for idx, table in enumerate(tables):
        header = [h.strip() for h in table[0].strip().strip('|').split('|')]
        rows = [list(map(str.strip, row.strip().strip('|').split('|'))) for row in table[2].strip().split('\n')]
        df = pd.DataFrame(rows, columns=header)
        print(f'Table {idx + 1}:')
        print(df)
        print()

# Example usage:
extract_tables_from_markdown('example.md')
